# Result figures for *Enhanced spatio-temporal electric load forecasts with less data using active deep learning*

---

## Overview
1. Test hyper parameters
2. Numerical results
3. Training and validation losses
4. Heuristics subsampling
5. Heuristics points per cluster
6. Heuristics query by coordinate
7. Query sequence importance
8. Manuscript figure: Results summary
9. Manuscript figure: Heuristics summary

In this notebook session, we summarize and visualize the results of our experiments. First test if all numerical results were computed with the same hyper parameters. Next, we plot our results and exemplar predictions so as to visually see and evaluate their meaning. Lastly, we create figures for our manuscript from experiments that best visualize our findings. We start with importing a number of packages that we use throughout this notebook session. 

In [1]:
import vis_results

HYPER_VIS = vis_results.HyperParameterVisualizing()

## 1. Test hyper parameters

Here, we import the results and the hyper parameters that were used in each of our experiment. We check whether all imported results are calculated on the exact same hyper parameters for the hypothesis test.

In [2]:
vis_results.test_hyper(HYPER_VIS)

## 2. Numeric results

In this section, we conclude the percentage of the data budget that is used, the percentage of novel senors in the queried candidate data point that is selected and the testing losses for each of our conducted experiments.

In [3]:
vis_results.show_numerical_results(HYPER_VIS)

profiles_100
delta1_valup1


Unnamed: 0,comp_fac,budget_usage,sensor_usage,test_loss,RF_loss,accuracy
spatio-temporal None PL train,1.0,100,100,1.317648,1.624579,19
spatio-temporal None PL val,1.0,100,100,1.317648,1.624579,19
spatio-temporal X_st rnd d_c train,4.7,100,100,0.425527,1.624579,74
spatio-temporal X_st rnd d_c val,4.7,100,100,0.425527,1.624579,74
spatio-temporal X_st min d_c train,11.6,100,100,0.668999,1.624579,59
spatio-temporal X_st min d_c val,11.6,100,100,0.668999,1.624579,59
spatio-temporal X_st max d_c train,2.4,100,100,0.397463,1.624579,76
spatio-temporal X_st max d_c val,2.4,100,100,0.397463,1.624579,76
spatio-temporal X_st avg d_c train,2.1,100,100,0.381471,1.624579,77
spatio-temporal X_st avg d_c val,2.1,100,100,0.381471,1.624579,77


profiles_100
delta1_valup0


Unnamed: 0,comp_fac,budget_usage,sensor_usage,test_loss,RF_loss,accuracy
spatio-temporal None PL train,1.0,100,100,1.752188,2.619218,33
spatio-temporal None PL val,1.0,100,100,1.752188,2.619218,33
spatio-temporal X_st rnd d_c train,5.1,100,100,0.689054,2.619218,74
spatio-temporal X_st rnd d_c val,5.1,100,100,0.689054,2.619218,74
spatio-temporal X_st min d_c train,15.2,100,100,0.857415,2.619218,67
spatio-temporal X_st min d_c val,15.2,100,100,0.857415,2.619218,67
spatio-temporal X_st max d_c train,7.4,100,100,0.811436,2.619218,69
spatio-temporal X_st max d_c val,7.4,100,100,0.811436,2.619218,69
spatio-temporal X_st avg d_c train,4.6,100,100,1.177447,2.619218,55
spatio-temporal X_st avg d_c val,4.6,100,100,1.177447,2.619218,55


profiles_100
delta0_valup0


Unnamed: 0,comp_fac,budget_usage,sensor_usage,test_loss,RF_loss,accuracy
spatio-temporal None PL train,1.0,81,100,2.075299,1.387907,0
spatio-temporal None PL val,1.0,81,100,2.075299,1.387907,0
spatio-temporal X_st rnd d_c train,2.9,55,100,1.460858,1.387907,0
spatio-temporal X_st rnd d_c val,2.9,55,100,1.460858,1.387907,0
spatio-temporal X_st min d_c train,2.8,55,100,1.052247,1.387907,24
spatio-temporal X_st min d_c val,2.8,55,100,1.052247,1.387907,24
spatio-temporal X_st max d_c train,2.9,36,88,0.786660,1.387907,43
spatio-temporal X_st max d_c val,2.9,36,88,0.786660,1.387907,43
spatio-temporal X_st avg d_c train,2.4,33,85,0.951458,1.387907,31
spatio-temporal X_st avg d_c val,2.4,33,85,0.951458,1.387907,31


profiles_100
delta0_valup1


Unnamed: 0,comp_fac,budget_usage,sensor_usage,test_loss,RF_loss,accuracy
spatio-temporal None PL train,1.0,81,100,2.106433,1.569501,0
spatio-temporal None PL val,1.0,81,100,2.106433,1.569501,0
spatio-temporal X_st rnd d_c train,4.5,63,100,1.194894,1.569501,24
spatio-temporal X_st rnd d_c val,4.5,63,100,1.194894,1.569501,24
spatio-temporal X_st min d_c train,3.9,63,100,1.309338,1.569501,17
spatio-temporal X_st min d_c val,3.9,63,100,1.309338,1.569501,17
spatio-temporal X_st max d_c train,3.9,39,98,1.166607,1.569501,26
spatio-temporal X_st max d_c val,3.9,39,98,1.166607,1.569501,26
spatio-temporal X_st avg d_c train,3.8,37,100,1.163170,1.569501,26
spatio-temporal X_st avg d_c val,3.8,37,100,1.163170,1.569501,26


## 3. Training and validation losses

For each prediction task, each query variable, each query variant and each experiment type, we create figures that allow us to compare their training and validation losses throughout the process of querying new candidate data points in each iteration of the algorithm that we propose.

In [4]:
%%capture 
vis_results.plot_train_val_hist(HYPER_VIS)

## 4. Heuristics subsampling


In [5]:
%%capture 
vis_results.plot_subsampling_heuristics(HYPER_VIS)

## 5. Heuristics points per cluster


In [6]:
%%capture 
vis_results.plot_pointspercluster_heuristics(HYPER_VIS)

## 6. Heuristics query by coordinate

In [7]:
%%capture 
vis_results.plot_querybycoordinate_heuristics(HYPER_VIS)

## 7. Query sequence importance

Here, we compare the training and validation losses of our active learning models against learning from the same data but in a randomized sequence. 

In [8]:
%%capture 
vis_results.plot_sequence_importance(HYPER_VIS)

## 8. Manuscript figure: Results summary


In [2]:
%%capture 
vis_results.plot_results_summary(HYPER_VIS)

## 9. Manuscript figure: Heuristics summary

In [3]:
%%capture 
vis_results.plot_results_summary(HYPER_VIS, 'heuristic_summary')